In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
from tensorflow import keras
import tensorflow.keras.layers as layers

D:\Programs\Python\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
vidObj = cv.VideoCapture("monuments_comp.mp4")

count = 0

success = 1

while success:
    success, image = vidObj.read()
    if(count%10==0):
        cv.imwrite("testFrames/monuments_clip/frame%d.jpg" % count, image)

    count += 1

In [7]:
vidObj.release()

In [16]:
def show_img(img):
    cv.imshow("Image", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [2]:
new_model = tf.keras.models.load_model('saved_model/dynasty_hog_noncave_vgg16_model')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Dataset\monument_hog',
                                                 target_size = (128,128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 114 images belonging to 5 classes.


In [4]:
class_names = list(training_set.class_indices.keys())
print(class_names)

['Colonial', 'Dravidian', 'Indo-Saracenic', 'Kalinga', 'Mughal']


In [5]:
def get_hog(input_img):
    img= np.float32(input_img) / 255.0

    gx = cv.Sobel(img, cv.CV_32F, 1, 0, ksize=1)
    gy = cv.Sobel(img, cv.CV_32F, 0, 1, ksize=1)

    mag, angle = cv.cartToPolar(gx, gy, angleInDegrees=True)
    mag = cv.resize(mag,(128,128))
    img_output = mag * 255.0
    img_output = img_output.astype('uint8')
    return img_output

In [12]:
import os
from pathlib import Path

paths = sorted(Path(r"D:\Projects\J Comps\Robo Vision\testFrames\monuments_clip").iterdir(), key=os.path.getmtime)

In [6]:
def get_prediction(img):
    img = cv.imread(img)
    img = get_hog(img)
#     cv.imshow("image",img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    img_array = tf.expand_dims(img, 0) # Create a batch
    print(img_array.shape)
    predictions =  new_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    output = class_names[np.argmax(score)]
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    return output

In [7]:
infer_img = get_prediction(str(paths[0]))

(1, 128, 128, 3)
1/1 [==============================] - 11s 11s/step
This image most likely belongs to Indo-Saracenic with a 28.74 percent confidence.


In [8]:
infer_img

'Indo-Saracenic'

In [44]:
similar_imgs = get_similarity(str(paths[0]), infer_img)

{0.15402387370042356: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_1 (2).jpg', 0.011420740063956145: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Mughal\\Image_2 (3).jpg', 0.05081946385465633: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_1 (7).jpg', 0.046783625730994156: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_1.jpg', 0.16556291390728478: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_1.png', 0.09136592051164916: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_2 (2).JPG', 0.061690314620604564: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_2 (4).jpg', 0.7042253521126761: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_2 (6).jpg', 0.01729804532087874: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Colonial\\Image_2 (7).jpg', 0.0: 'D:\\Projects\\J Comps\\Robo Vision\\Dataset\\dynasty\\Dravidian\\

1.8590998043052838
1.7808219178082192


In [45]:
print(len(similar_imgs))

3


In [46]:
out = cv.VideoWriter('output_video_similarity.avi',cv.VideoWriter_fourcc(*'DIVX'), 60, (512,512))

In [47]:
for i in similar_imgs:
    cv.imshow("Image",i)
    for j in range(60):
        out.write(i)
    cv.waitKey(1)
cv.destroyAllWindows()

In [48]:
out.release()

In [19]:
infer_images = []

In [20]:
print(len(infer_images))

0


In [21]:
out = cv.VideoWriter('output_video_compilation.avi',cv.VideoWriter_fourcc(*'DIVX'), 60, (512,512))

In [24]:
for i in infer_images:
    cv.imshow("Image",i)
    out.write(i)
    cv.waitKey(1)
cv.destroyAllWindows()

In [25]:
out.release()

In [23]:
for p in paths:
    output = get_prediction(str(p))
#     output =  "Test"
    font = cv.FONT_HERSHEY_SIMPLEX
    org = (100,100)
    fontScale = 3
    color = (0, 0, 0)
    thickness = 2
    img = cv.imread(str(p))
    print(output)
    image = cv.putText(img, output, org, font,fontScale, color, thickness, cv.LINE_AA)
    mag = cv.resize(image,(512,512))
    infer_images.append(mag)
#     cv.imshow('Frame', mag)
#     if cv.waitKey(25) & 0xFF == ord('q'):
#             break
#     get_similarity(str(p),output)

(1, 128, 128, 3)
1/1 [==============================] - 0s 18ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 17ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 

Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 17ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 26.60 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 26.56 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 31ms/step
This image most likely belongs to Indo-Saracenic with a 25.28 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 17ms/step
This image most likely belongs to Indo-Saracenic with a 26.71 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 26ms/step
This image most likely belongs to Indo-Saracenic with a 25.53 percent confidence.
Indo

Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Indo-Saracenic with a 27.75 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 26.79 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 18ms/step
This image most likely belongs to Indo-Saracenic with a 26.79 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 26.83 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Indo-Saracenic with a 27.19 percent confidence.
Indo

1/1 [==============================] - 0s 24ms/step
This image most likely belongs to Colonial with a 24.36 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 23.05 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Colonial with a 24.89 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 18ms/step
This image most likely belongs to Colonial with a 23.86 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Indo-Saracenic with a 26.89 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 25ms/step
This image most likely belongs to Colonial with a 26.76 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 42ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 16ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 40ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 40ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely b

...

1/1 [==============================] - 0s 22ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 23ms/step
This image most likely belongs to Indo-Saracenic with a 26.75 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 23ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 28ms/step
This image most likely belongs to Indo-Saracenic with a 26.40 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 448ms/step
This image most likely belongs to Indo-Saracenic with a 26.53 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 26ms/step
This image most likely belongs to Indo-Saracenic with a 26.82 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==========

1/1 [==============================] - 0s 26ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 27ms/step
This image most likely belongs to Indo-Saracenic with a 26.80 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 24ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 28ms/step
This image most likely belongs to Indo-Saracenic with a 26.80 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 23ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 24ms/step
This image most likely belongs to Indo-Saracenic with a 26.76 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 

Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 21ms/step
This image most likely belongs to Colonial with a 27.01 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 18ms/step
This image most likely belongs to Colonial with a 26.78 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 23ms/step
This image most likely belongs to Colonial with a 26.79 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 23ms/step
This image most likely belongs to Colonial with a 26.98 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Colonial with a 27.00 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Colonial with a 26.87 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/ste

1/1 [==============================] - 0s 21ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 30ms/step
This image most likely belongs to Colonial with a 26.77 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Colonial with a 26.79 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 21ms/step
This image most likely belongs to Colonial with a 26.84 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Colonial with a 26.90 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 28ms/step
This image most likely belongs to Colonial with a 27.00 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 23ms/step
This image m

Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 26ms/step
This image most likely belongs to Indo-Saracenic with a 25.16 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Indo-Saracenic with a 27.90 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 21ms/step
This image most likely belongs to Indo-Saracenic with a 26.30 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Colonial with a 26.75 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 20ms/step
This image most likely belongs to Colonial with a 26.76 percent confidence.
Colonial
(1, 128, 128, 3)
1/1 [==============================] - 0s 27ms/step
This image most likely belongs to Indo-Saracenic with a 26.77 percent confidence.
Indo-Saracenic
(1, 128, 128, 3)
1/

## Get similarity

In [14]:
def filter_match(matches):
    good_points = []
    ratio = 0.6
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good_points.append(m)
    return good_points

In [15]:
def percentage(img,img_2):
    sift = cv.SIFT_create()
    kp_1, desc_1 = sift.detectAndCompute(img, None)
    kp_2, desc_2 = sift.detectAndCompute(img_2, None)
    number_keypoints = 0
    if len(kp_1) <= len(kp_2):
        number_keypoints = len(kp_1)
    else:
        number_keypoints = len(kp_2)
    #print("Keypoints 1ST Image: " + str(len(kp_1)))
    #print("Keypoints 2ND Image: " + str(len(kp_2)))
    index_params = dict(algorithm=0, trees=5)
    search_params = dict()
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc_1, desc_2, k=2)
    good_points=filter_match(matches)
    percent=len(good_points) / number_keypoints * 100
    return percent
    #result = cv2.drawMatches(img, kp_1, img_2, kp_2, good_points, None)


In [16]:
def get_similarity(img,output):
    # output = "Colonial"
    img = cv.imread(img)
    f = r"D:\Projects\J Comps\Robo Vision\Dataset\dynasty"
    perc=dict()

    for cats in os.listdir(f):
        if cats==output:
            continue
        path=f+"\\"+cats
        for imgs in os.listdir(path):
            f_img= path+"\\"+imgs
            img_2=cv.imread(f_img,0)
            #print(f_img)
            p=percentage(img,img_2)
            perc[p]=f_img
            #plt.imshow(img_2)
    print(perc)
    sorted_perc = sorted(perc.items(), key=lambda x:x[0],reverse=True)

    #print(sorted_perc)
    n=3
    count=0
    # img=cv2.imread(r"dataset\dynasty\Mughal\Image_1 (2).jpg")
    # show_img(img)
    similar_imgs = []
    for tup in sorted_perc:
        if count==n:
            break
        #plt.imshow(tup[1])
        print(tup[0])
        img=cv.imread(tup[1])
        show_img(img)
        img = cv.resize(img,(512,512))
        count=count+1
        similar_imgs.append(img)
    return similar_imgs